In [1]:
# from ipyleaflet import Map, Marker, Icon, CircleMarker, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
# from ipywidgets import HTML

import ipyleaflet
import geopandas
import json
import numpy as np
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import ipywidgets as ipyw

import map_methods
import math

In [48]:
mk_municipalities = geopandas.read_file("https://github.com/VikiPeeva/coronavirus-macedonia-tryouts/raw/master/podatoci/macedonia/municipalities/mk_municipalities.geojson")
mk_cities = mk_municipalities.dissolve(by='entity_mk', aggfunc='sum')                                
# mk_cities['entity_mk'] = mk_cities.index
mk_cities['id'] = mk_cities.index

In [49]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age', 'source'], header=0)

infected_df

,city,infected_in,date,count,age,source
0,Скопје,Италија,2020-02-26,1,50-59,https://a1on.mk/macedonia/prv-sluchaj-na-zabol...
1,Дебар,Италија,2020-03-06,2,60-69,http://zdravstvo.gov.mk/dva-novi-potvrdeni-slu...
2,Дебар,Дебар,2020-03-09,2,60-69,https://sitel.com.mk/dva-novi-sluchai-na-koron...
3,Скопје,Италија,2020-03-09,1,50-59,https://makfax.com.mk/makedonija/%D1%84%D0%B8%...
4,Дебар,Дебар,2020-03-09,1,40-49,https://telma.com.mk/ushte-tri-novi-sluchai-na...
5,Дебар,Дебар,2020-03-11,2,60-69,https://a1on.mk/macedonia/dvajca-novi-zaboleni...
6,Дебар,Дебар,2020-03-13,4,50-59,https://civilmedia.mk/13-mart-chetiri-novi-slu...
7,Скопје,Шпанија,3/14/2020,1,непознато,https://www.magazin.mk/%D0%BA%D0%BE%D1%80%D0%B...
8,Дебар,Дебар,3/14/2020,5,непознато,https://makfax.com.mk/makedonija/%d0%b2%d0%b8%...
9,Скопје,Холандија,3/16/2020,1,50-59,https://kurir.mk/makedonija/vesti/pet-novi-slu...


In [50]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age', 'source'], header=1)
healed_df

,city,date,count,age,source


In [5]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age', 'source'])
dead_df

,city,date,count,age,source


In [51]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
covid_mk = pd.merge(infected_by_city, healded_by_city, on='city', how='outer', suffixes=['_infected', '_healed'])
covid_mk['count'] = covid_mk.count_infected
covid_mk = pd.merge(covid_mk, dead_by_city, on='city', how='outer', suffixes=['', '_dead'])
covid_mk.fillna(int(0), inplace=True)
covid_mk['count'] = covid_mk.count_infected - covid_mk.count_healed - covid_mk.count_dead
covid_mk = covid_mk.astype(int)
covid_mk

,count_infected,count_healed,count,count_dead
city,,,,
Дебар,20,0,20,0
Скопје,7,0,7,0


In [52]:
covid_mk_cities = mk_cities.merge(covid_mk, left_on='id', right_on='city', how='left')
covid_mk_cities

,geometry,latitude,longitude,area_sqkm,area,population,scalerank,diss_me,adm0_sr,provnum_ne,...,gns_level,min_label,max_label,min_zoom,ne_id,id,count_infected,count_healed,count,count_dead
0,"POLYGON ((21.02797 42.02765, 21.02810 42.02753...",42.0311,21.1290,0,174.0,10.790,10,2963,1,20001,...,1,10.0,11.0,10.0,1159314547,Јегуновце,NaN,NaN,NaN,NaN
1,"POLYGON ((21.61108 42.08140, 21.61101 42.08098...",42.0185,21.5862,0,38.0,11.597,10,2919,1,20057,...,1,10.0,11.0,10.0,1159314449,Арачиново,NaN,NaN,NaN,NaN
2,"POLYGON ((22.78298 41.78510, 22.78360 41.78545...",41.6344,22.8027,0,597.0,13.941,10,3004,1,20029,...,1,10.0,11.0,10.0,1159314569,Берово,NaN,NaN,NaN,NaN
3,"POLYGON ((21.25219 40.89330, 21.24280 40.87920...",41.0402,21.2925,0,790.0,95.385,10,2896,1,20040,...,1,10.0,11.0,10.0,1159314401,Битола,NaN,NaN,NaN,NaN
4,"POLYGON ((22.52113 41.15881, 22.52180 41.15898...",41.1772,22.5841,0,114.0,8.707,10,2996,1,20025,...,1,10.0,11.0,10.0,1159314555,Богданци,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,"POLYGON ((20.66262 41.44884, 20.66553 41.44795...",41.4644,20.5799,0,107.0,6.519,10,2951,1,20036,...,1,10.0,11.0,10.0,1159314519,Центар Жупа,NaN,NaN,NaN,NaN
66,"POLYGON ((21.81383 41.49223, 21.81417 41.49180...",41.5807,21.5939,0,727.0,7.673,10,2922,1,20070,...,1,10.0,11.0,10.0,1159314457,Чашка,NaN,NaN,NaN,NaN
67,"POLYGON ((22.21717 41.85944, 22.21745 41.85988...",41.8644,22.2896,0,133.0,7.490,10,2925,1,20058,...,1,10.0,11.0,10.0,1159314463,Чешиново-Облешево,NaN,NaN,NaN,NaN
68,"POLYGON ((21.52347 42.19750, 21.52344 42.19761...",42.1353,21.3885,0,215.0,8.493,10,2926,1,20004,...,1,10.0,11.0,10.0,1159314465,Чучер Сандево,NaN,NaN,NaN,NaN


In [53]:
covid_mk_cities['count'] = covid_mk_cities['count'].fillna(0).astype(int)
covid_mk_cities['count_infected'] = covid_mk_cities['count_infected'].fillna(0).astype(int)
covid_mk_cities['count_healed'] = covid_mk_cities['count_healed'].fillna(0).astype(int)
covid_mk_cities['count_dead'] = covid_mk_cities['count_dead'].fillna(0).astype(int)

In [54]:
scale = np.log2((covid_mk_cities['count'] / (covid_mk_cities['population'] / 2000) + 1))
idd = map(str, covid_mk_cities.index)
covid_active = dict(zip(idd, scale))
covid_mk_cities_geo_json = json.loads(covid_mk_cities.to_json())

In [55]:
covid_mk_cities

,geometry,latitude,longitude,area_sqkm,area,population,scalerank,diss_me,adm0_sr,provnum_ne,...,gns_level,min_label,max_label,min_zoom,ne_id,id,count_infected,count_healed,count,count_dead
0,"POLYGON ((21.02797 42.02765, 21.02810 42.02753...",42.0311,21.1290,0,174.0,10.790,10,2963,1,20001,...,1,10.0,11.0,10.0,1159314547,Јегуновце,0,0,0,0
1,"POLYGON ((21.61108 42.08140, 21.61101 42.08098...",42.0185,21.5862,0,38.0,11.597,10,2919,1,20057,...,1,10.0,11.0,10.0,1159314449,Арачиново,0,0,0,0
2,"POLYGON ((22.78298 41.78510, 22.78360 41.78545...",41.6344,22.8027,0,597.0,13.941,10,3004,1,20029,...,1,10.0,11.0,10.0,1159314569,Берово,0,0,0,0
3,"POLYGON ((21.25219 40.89330, 21.24280 40.87920...",41.0402,21.2925,0,790.0,95.385,10,2896,1,20040,...,1,10.0,11.0,10.0,1159314401,Битола,0,0,0,0
4,"POLYGON ((22.52113 41.15881, 22.52180 41.15898...",41.1772,22.5841,0,114.0,8.707,10,2996,1,20025,...,1,10.0,11.0,10.0,1159314555,Богданци,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,"POLYGON ((20.66262 41.44884, 20.66553 41.44795...",41.4644,20.5799,0,107.0,6.519,10,2951,1,20036,...,1,10.0,11.0,10.0,1159314519,Центар Жупа,0,0,0,0
66,"POLYGON ((21.81383 41.49223, 21.81417 41.49180...",41.5807,21.5939,0,727.0,7.673,10,2922,1,20070,...,1,10.0,11.0,10.0,1159314457,Чашка,0,0,0,0
67,"POLYGON ((22.21717 41.85944, 22.21745 41.85988...",41.8644,22.2896,0,133.0,7.490,10,2925,1,20058,...,1,10.0,11.0,10.0,1159314463,Чешиново-Облешево,0,0,0,0
68,"POLYGON ((21.52347 42.19750, 21.52344 42.19761...",42.1353,21.3885,0,215.0,8.493,10,2926,1,20004,...,1,10.0,11.0,10.0,1159314465,Чучер Сандево,0,0,0,0


In [61]:
hospital_data = pd.read_csv("https://raw.githubusercontent.com/VikiPeeva/coronavirus-macedonia-tryouts/master/podatoci/AdditionalInfo-Hospitals.csv")

hospital_data['hospital'] = hospital_data.Hospital

In [62]:
sick_hospitals = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Хоспитализирани.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])

recovered_hospitals = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Оздравени по Болница.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])

dead_hospitals = pd.read_csv("../podatoci/Корона вирус податоци Македонија - Мртви по Болница.csv", header=0,
                 names=['hospital', "date", "count", "age_range"])


sick_hospital = sick_hospitals.groupby(["hospital"]).sum()
recovered_hospital = recovered_hospitals.groupby(["hospital"]).sum()
dead_hospital = dead_hospitals.groupby(["hospital"]).sum()

In [63]:
covid_hospitals = pd.merge(sick_hospital, recovered_hospital, on='hospital', how='outer')
covid_hospitals = pd.merge(covid_hospitals, dead_hospital, on='hospital', how='outer')
covid_hospitals.fillna(0, inplace=True)
covid_hospitals = pd.merge(covid_hospitals, hospital_data, on='hospital', how='inner')
covid_hospitals = covid_hospitals.set_index("hospital")
covid_hospitals

,count_x,count_y,date,count,age_range,Hospital,Latitude,Longitude
hospital,,,,,,,,
Клиника за инфективни болести Скопје,14,1.0,0.0,0.0,0.0,Клиника за инфективни болести Скопје,41.988005,21.425646
Општа болница Дебар,5,0.0,0.0,0.0,0.0,Општа болница Дебар,41.523717,20.523101


In [64]:
def hide_marker(name, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    return (infected - healed - dead) < 1

def get_marker(name, location, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    current = infected - healed - dead
    scale = math.log(current, 2)
    hospital_icon = ipyleaflet.Icon(
        icon_url='https://www.freeiconspng.com/uploads/ambulance-cross-hospital-icon-11.png',
        icon_size=(10*scale, 10*scale),
        icon_anchor=(5*scale, 5*scale),
        popup_anchor=(0, -5*scale)
    )
    return ipyleaflet.Marker(location=location, draggable=False, icon=hospital_icon, z_index_offset=1000)

In [65]:

layer = ipyleaflet.Choropleth(
    geo_data=covid_mk_cities_geo_json,
    choro_data=covid_active,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.4, 'weight': 0.05}
)

geo_data = ipyleaflet.GeoData(
    geo_dataframe = covid_mk_cities,
    style={'color': 'black', 'opacity':0.00, 'weight':0, 'dashArray':'2', 'fillOpacity':0.0},
    hover_style={'fillColor': 'grey' , 'fillOpacity': 0.3},
    name = 'Countries',
    
)

# def hover_handler(event=None, feature=None, id=None, properties=None):
#     label.value = properties['id'] + " " + str(properties['count'])

# geo_data.on_hover(hover_handler)
# label = ipyw.Label(layout=ipyw.Layout(width='100%'))
m = map_methods.get_macedonia_map()
m.add_layer(layer)
m.add_layer(geo_data)
map_methods.add_marker_layer(m=m, d=covid_hospitals, 
                             cols=["count_x", "count_y", "count"], 
                             names=["Заразени", "Излечени" ,"Мртви"],
                            get_marker=get_marker,
                            hide_marker=hide_marker,
                            lng='Longitude',
                            lat='Latitude')
m.add_control(ipyleaflet.LayersControl())

m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…